In [1]:
import numpy as np
import pickle
import math
import matplotlib.pyplot as plt
import pandas as pd
import time
import pprint
import seaborn as sns
import CMR_IA as cmr
sns.set_context('paper')
# pd.set_option('display.max_columns', None)

In [2]:
with open("../Data/simu6c_design.pkl", 'rb') as inp:
    df_study = pickle.load(inp)
    df_test = pickle.load(inp)
# df_study = df_study.loc[df_study.session < 500]
# df_test = df_test.loc[df_test.session < 500]

In [3]:
df_study

,study_itemno1,study_itemno2,study_item1,study_item2,session
0,140,838,BLUEBERRY,LOCK,0
1,1587,1240,WARDROBE,SAXOPHONE,0
2,1132,702,PROTON,HELMET,0
3,299,1282,CHILD,SHELTER,0
4,1391,1059,STRAWBERRY,PICKLE,0
...,...,...,...,...,...
11995,301,1184,CHIMNEY,REPORT,999
11996,57,965,ATTIRE,NUT,999
11997,155,1112,BOULDER,POSSUM,999
11998,106,1368,BEAR,STAKE,999


In [4]:
df_test

,test_itemno,test_item,correct_ans,order,rep,test,session,test_pos,pair_idx
0,210,CABBAGE,327,1,1,1,0,1,11
1,1251,SCOTCH,819,2,1,1,0,2,8
2,145,BODY,1198,1,1,1,0,3,7
3,1059,PICKLE,1391,2,1,1,0,4,5
4,711,HOE,1519,1,1,1,0,5,9
...,...,...,...,...,...,...,...,...,...
23995,965,NUT,57,2,1,2,999,20,11997
23996,1075,PISTOL,644,2,1,2,999,21,11993
23997,1090,PLUM,1609,2,1,2,999,22,11991
23998,1184,REPORT,301,2,1,2,999,23,11996


### Run CMR-IA

In [5]:
s_mat = np.load('../Data/wordpools/ltp_FR_similarity_matrix.npy')

In [6]:
params = cmr.make_default_params()
params.update(
    beta_enc = 0.5,
    beta_rec = 0.45, # beta for cue
    beta_rec_new = 0.1, # beta for retrieved item
    beta_rec_post = 0.1,
    gamma_fc = 0.5,
    gamma_cf = 0.5,
    s_fc = 0.1,
    s_cf = 0.1,
    c_thresh = 0.70,
    c_thresh_ass = 0.01,
    a = 2800,
    b = 2.14,
    m = 0,
    n = 1,
    kappa = 0.02,
    lamb = 0.01,
    eta = 0.01,
    phi_s = 1,
    phi_d = 0.6,
    nitems_in_accumulator = 10,
    d_ass = 1,
    learn_while_retrieving = True,
    omega = 100
)
params

{'beta_enc': 0.5,
 'beta_rec': 0.45,
 'beta_rec_new': 0.1,
 'beta_rec_post': 0.1,
 'beta_distract': None,
 'phi_s': 1,
 'phi_d': 0.6,
 's_cf': 0.1,
 's_fc': 0.1,
 'kappa': 0.02,
 'eta': 0.01,
 'omega': 100,
 'alpha': 4,
 'c_thresh': 0.7,
 'c_thresh_ass': 0.01,
 'd_ass': 1,
 'lamb': 0.01,
 'rec_time_limit': 60000.0,
 'dt': 10,
 'nitems_in_accumulator': 10,
 'max_recalls': 50,
 'learn_while_retrieving': True,
 'a': 2800,
 'b': 2.14,
 'm': 0,
 'n': 1,
 'c1': 0,
 'gamma_fc': 0.5,
 'gamma_cf': 0.5}

In [7]:
df_simu, f_in = cmr.run_norm_cr_multi_sess(params,df_study, df_test ,s_mat)
df_simu['test_pos'] = np.tile(np.arange(1,25),1000)
df_simu

CMR2 Time: 22.991123914718628


,session,test_itemno,s_resp,s_rt,csim,test_pos
0,0,210,1372.0,1220.0,0.812490,1
1,0,1251,327.0,1410.0,0.830015,2
2,0,145,1198.0,1440.0,0.831033,3
3,0,1059,819.0,1490.0,0.701293,4
4,0,711,1519.0,1450.0,0.848601,5
...,...,...,...,...,...,...
23995,999,965,-2.0,-2.0,0.624729,20
23996,999,1075,644.0,650.0,0.845576,21
23997,999,1090,-1.0,-1.0,-1.000000,22
23998,999,1184,-1.0,-1.0,-1.000000,23


In [8]:
df_simu = df_simu.merge(df_test,on=['session','test_itemno','test_pos'])
df_simu['correct'] = df_simu.s_resp == df_simu.correct_ans
df_simu

,session,test_itemno,s_resp,s_rt,csim,test_pos,test_item,correct_ans,order,rep,test,pair_idx,correct
0,0,210,1372.0,1220.0,0.812490,1,CABBAGE,327,1,1,1,11,False
1,0,1251,327.0,1410.0,0.830015,2,SCOTCH,819,2,1,1,8,False
2,0,145,1198.0,1440.0,0.831033,3,BODY,1198,1,1,1,7,True
3,0,1059,819.0,1490.0,0.701293,4,PICKLE,1391,2,1,1,5,False
4,0,711,1519.0,1450.0,0.848601,5,HOE,1519,1,1,1,9,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23995,999,965,-2.0,-2.0,0.624729,20,NUT,57,2,1,2,11997,False
23996,999,1075,644.0,650.0,0.845576,21,PISTOL,644,2,1,2,11993,True
23997,999,1090,-1.0,-1.0,-1.000000,22,PLUM,1609,2,1,2,11991,False
23998,999,1184,-1.0,-1.0,-1.000000,23,REPORT,301,2,1,2,11996,False


In [9]:
df_cond = df_simu.groupby(["pair_idx","test"])['order'].mean().to_frame(name='corr_rate').reset_index()
df_cond = df_cond.pivot_table(index='pair_idx',columns='test',values='corr_rate').reset_index()
df_cond.columns = ['pair_idx','test1','test2']
def cond(x):
    test1 = x['test1']
    test2 = x['test2']
    if test1 == 1 and test2 == 1:
        return 'F-F'
    elif test1 == 1 and test2 == 2:
        return 'F-B'
    elif test1 == 2 and test2 == 1:
        return 'B-F'
    elif test1 == 2 and test2 == 2:
        return 'B-B'

df_cond['cond'] = df_cond.apply(lambda x:cond(x),axis=1)
df_cond['cong'] = df_cond.apply(lambda x: 'Identical' if x['cond'] == 'F-F' or x['cond'] == 'B-B' else 'Reversed',axis=1)
df_cond

,pair_idx,test1,test2,cond,cong
0,1,2.0,1.0,B-F,Reversed
1,2,1.0,2.0,F-B,Reversed
2,3,2.0,1.0,B-F,Reversed
3,4,1.0,1.0,F-F,Identical
4,5,2.0,2.0,B-B,Identical
...,...,...,...,...,...
11995,11996,2.0,2.0,B-B,Identical
11996,11997,1.0,2.0,F-B,Reversed
11997,11998,2.0,1.0,B-F,Reversed
11998,11999,1.0,1.0,F-F,Identical


In [10]:
pairidx2cond = df_cond.loc[:,['pair_idx','cond']].set_index("pair_idx").to_dict()['cond']
pairidx2cong = df_cond.loc[:,['pair_idx','cong']].set_index("pair_idx").to_dict()['cong']
df_simu['cond'] = df_simu.apply(lambda x:pairidx2cond[x['pair_idx']],axis=1)
df_simu['cong'] = df_simu.apply(lambda x:pairidx2cong[x['pair_idx']],axis=1)
df_simu.head(24)


,session,test_itemno,s_resp,s_rt,csim,test_pos,test_item,correct_ans,order,rep,test,pair_idx,correct,cond,cong
0,0,210,1372.0,1220.0,0.812490,1,CABBAGE,327,1,1,1,11,False,F-F,Identical
1,0,1251,327.0,1410.0,0.830015,2,SCOTCH,819,2,1,1,8,False,B-B,Identical
2,0,145,1198.0,1440.0,0.831033,3,BODY,1198,1,1,1,7,True,F-F,Identical
3,0,1059,819.0,1490.0,0.701293,4,PICKLE,1391,2,1,1,5,False,B-B,Identical
4,0,711,1519.0,1450.0,0.848601,5,HOE,1519,1,1,1,9,True,F-B,Reversed
5,0,838,-1.0,-1.0,-1.000000,6,LOCK,140,2,1,1,1,False,B-F,Reversed
6,0,153,-1.0,-1.0,-1.000000,7,BOTTLE,736,1,1,1,6,False,F-B,Reversed
7,0,1587,-1.0,-1.0,-1.000000,8,WARDROBE,1240,1,1,1,2,False,F-B,Reversed
8,0,856,-1.0,-1.0,-1.000000,9,MAID,607,2,1,1,10,False,B-B,Identical
9,0,702,-1.0,-1.0,-1.000000,10,HELMET,1132,2,1,1,3,False,B-F,Reversed


In [11]:
len(f_in[0])

24

In [12]:
sessions = np.unique(df_simu.session.to_numpy())
tmp_corr_fin = []
tmp_omean_fin = []
tmp_omax_fin = []

for sess in sessions:
    df_tmp = df_study.loc[df_study.session == sess]
    tmp1 = df_tmp.study_itemno1.to_numpy()
    tmp2 = df_tmp.study_itemno2.to_numpy()
    tmp = np.concatenate((tmp1, tmp2))
    tmp = np.sort(tmp)

    df_tmp1 = df_simu.loc[df_simu.session == sess]
    tmp_test = df_tmp1.test_itemno
    tmp_corr = df_tmp1.correct_ans
    # tmp_test = df_simu.loc[df_simu.session == sess, 'test_itemno']
    # tmp_corr = df_simu.loc[df_simu.session == sess, 'correct_ans']
    testid = np.searchsorted(tmp, tmp_test)
    corrid = np.searchsorted(tmp, tmp_corr)

    fin_sess = f_in[sess]
    # tmp_corr_fin = []
    # tmp_omean_fin = []
    # tmp_omax_fin = []
    for i in range(len(testid)):
        tmp_corr_fin.append(fin_sess[i][testid[i]])
        tmp_omean_fin.append(np.mean(np.delete(fin_sess[i], [testid[i],corrid[i]])))
        tmp_omax_fin.append(np.delete(fin_sess[i], [testid[i],corrid[i]]).max())
    # df_simu.loc[df_simu.session == sess, 'corr_fin']= tmp_corr_fin
    # df_simu.loc[df_simu.session == sess, 'omean_fin'] = tmp_omean_fin
    # df_simu.loc[df_simu.session == sess, 'omax_fin'] = tmp_omax_fin

df_simu['corr_fin']= tmp_corr_fin
df_simu['omean_fin'] = tmp_omean_fin
df_simu['omax_fin'] = tmp_omax_fin
df_simu

,session,test_itemno,s_resp,s_rt,csim,test_pos,test_item,correct_ans,order,rep,test,pair_idx,correct,cond,cong,corr_fin,omean_fin,omax_fin
0,0,210,1372.0,1220.0,0.812490,1,CABBAGE,327,1,1,1,11,False,F-F,Identical,0.860396,0.377403,0.739139
1,0,1251,327.0,1410.0,0.830015,2,SCOTCH,819,2,1,1,8,False,B-B,Identical,0.714210,0.507152,1.467383
2,0,145,1198.0,1440.0,0.831033,3,BODY,1198,1,1,1,7,True,F-F,Identical,0.732686,0.588213,1.429066
3,0,1059,819.0,1490.0,0.701293,4,PICKLE,1391,2,1,1,5,False,B-B,Identical,0.703359,0.662693,1.481276
4,0,711,1519.0,1450.0,0.848601,5,HOE,1519,1,1,1,9,True,F-B,Reversed,0.762233,0.718607,1.439292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23995,999,965,-2.0,-2.0,0.624729,20,NUT,57,2,1,2,11997,False,F-B,Reversed,2.145970,1.065169,2.046029
23996,999,1075,644.0,650.0,0.845576,21,PISTOL,644,2,1,2,11993,True,B-B,Identical,1.711991,1.001381,1.755553
23997,999,1090,-1.0,-1.0,-1.000000,22,PLUM,1609,2,1,2,11991,False,B-B,Identical,1.595298,1.072480,2.336833
23998,999,1184,-1.0,-1.0,-1.000000,23,REPORT,301,2,1,2,11996,False,B-B,Identical,0.715178,1.132385,2.134304


In [13]:
df_fin = df_simu.groupby(['cong','test'])['corr_fin','omean_fin','omax_fin'].mean().reset_index()
df_fin['dif'] = df_fin['corr_fin'] - df_fin['omax_fin']
df_fin

C:\Users\beige\AppData\Local\Temp\ipykernel_13292\1283200413.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_fin = df_simu.groupby(['cong','test'])['corr_fin','omean_fin','omax_fin'].mean().reset_index()


,cong,test,corr_fin,omean_fin,omax_fin,dif
0,Identical,1,0.737718,0.654445,1.299161,-0.561443
1,Identical,2,1.171474,0.886003,1.956208,-0.784733
2,Reversed,1,0.737063,0.654397,1.302453,-0.565390
3,Reversed,2,1.113449,0.879231,1.938412,-0.824964


### PLOT

In [14]:
df_simu.head(24)

,session,test_itemno,s_resp,s_rt,csim,test_pos,test_item,correct_ans,order,rep,test,pair_idx,correct,cond,cong,corr_fin,omean_fin,omax_fin
0,0,210,1372.0,1220.0,0.812490,1,CABBAGE,327,1,1,1,11,False,F-F,Identical,0.860396,0.377403,0.739139
1,0,1251,327.0,1410.0,0.830015,2,SCOTCH,819,2,1,1,8,False,B-B,Identical,0.714210,0.507152,1.467383
2,0,145,1198.0,1440.0,0.831033,3,BODY,1198,1,1,1,7,True,F-F,Identical,0.732686,0.588213,1.429066
3,0,1059,819.0,1490.0,0.701293,4,PICKLE,1391,2,1,1,5,False,B-B,Identical,0.703359,0.662693,1.481276
4,0,711,1519.0,1450.0,0.848601,5,HOE,1519,1,1,1,9,True,F-B,Reversed,0.762233,0.718607,1.439292
5,0,838,-1.0,-1.0,-1.000000,6,LOCK,140,2,1,1,1,False,B-F,Reversed,0.647131,0.762737,1.412432
6,0,153,-1.0,-1.0,-1.000000,7,BOTTLE,736,1,1,1,6,False,F-B,Reversed,0.739556,0.744879,1.273071
7,0,1587,-1.0,-1.0,-1.000000,8,WARDROBE,1240,1,1,1,2,False,F-B,Reversed,0.672303,0.710885,1.106639
8,0,856,-1.0,-1.0,-1.000000,9,MAID,607,2,1,1,10,False,B-B,Identical,0.713482,0.750870,1.216762
9,0,702,-1.0,-1.0,-1.000000,10,HELMET,1132,2,1,1,3,False,B-F,Reversed,0.672935,0.717474,1.054311


In [15]:
dfgp_test_order = df_simu.groupby(['rep','test','order']).correct.mean().to_frame(name='corr_rate').reset_index()
dfgp_test_order

,rep,test,order,corr_rate
0,1,1,1,0.356667
1,1,1,2,0.348667
2,1,2,1,0.337000
3,1,2,2,0.317167


In [24]:
def contingency_tab(x):
    tmp_x = pd.pivot_table(x,index='pair_idx',columns='test', values= 'correct')
    test1 = pd.Categorical(tmp_x[1], categories=[1, 0])
    test2 = pd.Categorical(tmp_x[2], categories=[1, 0])
    return pd.crosstab(index=test2,columns=test1,normalize='all')

In [25]:
dfgp_rep_cong = df_simu.groupby(['cong']).apply(contingency_tab)
dfgp_rep_cong

col_0                   1         0
cong      row_0                    
Identical 1      0.294167  0.022167
          0      0.058833  0.624833
Reversed  1      0.293667  0.044167
          0      0.058667  0.603500

In [18]:
df_simu.groupby(['cong','test']).correct.mean()

cong       test
Identical  1       0.353000
           2       0.316333
Reversed   1       0.352333
           2       0.337833
Name: correct, dtype: float64

In [34]:
def Yule_Q(A, B, C, D):
    return (A * D - B * C) / (A * D + B * C)

In [39]:
tab_identical = dfgp_rep_cong.loc['Identical']
Yule_Q(tab_identical[1][1],tab_identical[0][1],tab_identical[1][0],tab_identical[0][0])

0.9859095243140164

In [43]:
tab_reversed = dfgp_rep_cong.loc['Reversed']
Yule_Q(tab_reversed[1][1],tab_reversed[0][1],tab_reversed[1][0],tab_reversed[0][0])

0.9711808884306777

In [41]:
Yule_Q(0.319, 0.012, 0.006, 0.663)

0.9993193709853524

In [42]:
Yule_Q(0.293, 0.122, 0.049, 0.537)

0.9267935757627709